In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "answerdotai/ModernBERT-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
modernBert = AutoModel.from_pretrained(model_id).to(device)
modernBert.eval()

def extract_embeddings(texts, model, tokenizer, method="cls", batch_size=16, max_length=512):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc=f"Extracting ({method})"):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
        with torch.no_grad():
            output = model(**enc).last_hidden_state

        if method == "cls":
            batch_emb = output[:, 0, :]
        elif method == "mean":
            batch_emb = output.mean(dim=1)
        elif method == "max":
            batch_emb = output.max(dim=1).values
        elif method == "cls+mean":
            batch_emb = torch.cat([output[:, 0, :], output.mean(dim=1)], dim=1)
        else:
            raise ValueError("Unsupported pooling method")

        embeddings.extend(batch_emb.cpu().numpy())
    return np.array(embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

In [2]:
def evaluate_split(train_csv, val_csv, label_col, pooling="cls+mean"):
    df_train = pd.read_csv(train_csv)
    df_val = pd.read_csv(val_csv)

    X_train = extract_embeddings(df_train["response"].tolist(), modernBert, tokenizer, method=pooling)
    X_val = extract_embeddings(df_val["response"].tolist(), modernBert, tokenizer, method=pooling)

    y_train = df_train[label_col].values
    y_val = df_val[label_col].values

    clf = LogisticRegression(max_iter=2000, class_weight="balanced")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)

    f1 = f1_score(y_val, y_pred, average="macro")
    acc = accuracy_score(y_val, y_pred)

    print(f"{label_col} [{pooling}] → Acc: {acc:.4f}, Macro F1: {f1:.4f}")
    return acc, f1


In [4]:
tasks = [
    "mistake_identification",
    "mistake_location",
    "providing_guidance",
    "actionability",
]
poolings = ["cls", "mean", "max", "cls+mean"]
results = {}

for task_name in tasks:
  for pooling in poolings:
    print(f"\n Evaluating {task_name}")
    acc, f1 = evaluate_split(
        train_csv=f"{task_name}_train.csv",
        val_csv=f"{task_name}_val.csv",
        label_col=task_name,
        pooling=pooling
    )
    results[task_name] = (acc, f1)



 Evaluating mistake_identification


Extracting (cls):   0%|          | 0/124 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
Extracting (cls): 100%|██████████| 31/31 [00:02<00:00, 10.78it/s]


mistake_identification [cls] → Acc: 0.8012, Macro F1: 0.6265

 Evaluating mistake_identification


Extracting (mean): 100%|██████████| 31/31 [00:02<00:00, 10.83it/s]


mistake_identification [mean] → Acc: 0.8235, Macro F1: 0.6558

 Evaluating mistake_identification


...

actionability [mean] → Acc: 0.6222, Macro F1: 0.5665

 Evaluating actionability


Extracting (max): 100%|██████████| 31/31 [00:02<00:00, 10.91it/s]


actionability [max] → Acc: 0.5899, Macro F1: 0.5313

 Evaluating actionability


Extracting (cls+mean): 100%|██████████| 31/31 [00:02<00:00, 10.91it/s]


actionability [cls+mean] → Acc: 0.5899, Macro F1: 0.5158
